# Gaussian Processes and infinite Neural Networks

## Multinomial Gaussian distributions

Before we discuss Gaussian processes, let us discuss the multinomial Gaussian distribution, which is defined as

$$\mathcal{N}(x | \mu, \Sigma) = \frac{1}{\sqrt{ (2 \pi)^d |\Sigma|}} \exp \left\{ - \frac{1}{2} (x - \mu)^T \, \Sigma^{-1} \, (x - \mu)\right\}.$$

The parameters of the distribution are the mean $\mu \in \mathbb{R}^d$ and the covariance matrix $\Sigma \in \mathbb{R}^{d \times d}$. In fact, we can restrict our analysis to symmetric positive definite covariance matrices $\Sigma$, without loss of generality.

Let us discuss conditional distributions. Assume $X \sim \mathcal{N}(x | \mu, \Sigma)$ and we have observed some components of $X$. It is common practice to distinguish between random variables denoted by capital letters and their actual realizations denoted by lowercase letters.